<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [110]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) #connecting to databse
c = conn.cursor() #whenever we want to execute the command

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [111]:
c.execute('DROP TABLE houses;') #doesnt work because no table call houses

# Save (commit) the changes:
conn.commit() 

In [112]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [113]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [114]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 

conn.commit()

Select all rows from houses

In [115]:
# A
result_allrows = c.execute('SELECT * from houses')
conn.commit()
result_allrows.fetchall()

[]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [116]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [117]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [118]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [119]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [120]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000]) #covert the input to an array 
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [121]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')

conn.commit()

QUIZ: What would `DELETE FROM houses` do?

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [122]:
# ANSWER
result_4 = c.execute('Select * FROM houses WHERE bdrms = 4')
result_4.fetchall()

[(5, 3000, 4, 75, 539900),
 (6, 1985, 4, 61, 299900),
 (11, 1940, 4, 7, 239999),
 (16, 2300, 4, 77, 449900),
 (19, 2609, 4, 5, 499998),
 (20, 3031, 4, 21, 599000),
 (24, 1962, 4, 53, 259900),
 (33, 2040, 4, 75, 314900),
 (35, 1811, 4, 24, 285900),
 (38, 2132, 4, 28, 345000),
 (39, 4215, 4, 66, 549000),
 (40, 2162, 4, 43, 287000),
 (43, 2567, 4, 57, 314000),
 (46, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [123]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1814.1923076923076, 323465.3076923077),
 (4, 2400.4285714285716, 377449.78571428574),
 (5, 4478.0, 699900.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [124]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [125]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [126]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [127]:
#ANSWER
results = c.execute("SELECT bdrms, AVG(price) AS avg_price FROM houses_pandas GROUP BY bdrms")
results.fetchall()

[(1, 169900.0),
 (2, 280866.6666666667),
 (3, 326403.92),
 (4, 377449.78571428574),
 (5, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [128]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [129]:
# Reading CSV to Dataframe
orders = pd.read_csv('P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('P12-OrderBreakdown.csv', encoding = 'utf-8')

In [130]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [131]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [132]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [133]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes 

In [134]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [135]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [136]:
# Establishing Local DB connection
#db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')
#tosql
# Save these two dataframes as a table in sqlite

In [152]:
sqlite_db = 'orders.db'
db_connection = sqlite3.connect(sqlite_db) #connecting to databse
#c = conn.cursor() #whenever we want to execute the command
orders.to_sql('orders', db_connection, if_exists='replace', index = False)
orders_break_down.to_sql('ORDERS_BREAKDOWN', db_connection, if_exists='replace', index = False) 

In [163]:
query = """
SELECT name FROM sqlite_master
WHERE type IN ('table','view') AND name NOT LIKE 'sqlite_%'
UNION ALL
SELECT name FROM sqlite_temp_master
WHERE type IN ('table','view')
ORDER BY 1;
"""
tables = sql.read_sql(query, con=db_connection)
tables

,name
0,ORDERS_BREAKDOWN
1,orders


#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [154]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM orders
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con = db_connection)
orders_head_10
#print(db_connection)

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [159]:
# Select Number of Orders for Each Customer
#query = """
        #SELECT * 
        #FROM orders
        #group by customer_name;
        #"""
query = """SELECT customer_name, count(order_id)
FROM orders
group by customer_name;"""
        
orders_no_order_customer = sql.read_sql(query, con = db_connection)
orders_no_order_customer 



,customer_name,count(order_id)
0,Aaron Bergman,11
1,Aaron Hawkins,7
2,Aaron Smayling,12
3,Adam Bellavance,4
4,Adam Hart,8
...,...,...
787,Xylona Preis,2
788,Yana Sorensen,5
789,Yoseph Carroll,11
790,Zuschuss Carroll,9


#### 6. Select Number of Customers for Each Country

In [161]:
# Select Number of Customers for Each Country
query = """SELECT country, count(customer_name)
FROM orders
group by country;"""
        
orders_no_country = sql.read_sql(query, con = db_connection)
orders_no_country

,country,count(customer_name)
0,Austria,135
1,Belgium,68
2,Denmark,29
3,Finland,34
4,France,991
5,Germany,806
6,Ireland,50
7,Italy,493
8,Netherlands,194
9,Norway,37


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [168]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table

query = """SELECT discount, sales, quantity
FROM ORDERS_BREAKDOWN
"""
        
orders_breakdown_table = sql.read_sql(query, con = db_connection)
orders_breakdown_table

,discount,sales,quantity
0,0.5,45.0,3
1,0.0,854.0,7
2,0.0,140.0,3
3,0.5,27.0,2
4,0.5,17.0,2
...,...,...,...
8042,0.0,245.0,2
8043,0.5,30.0,2
8044,0.5,23.0,4
8045,0.5,108.0,3


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [174]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table
query = """SELECT discount, sales, quantity, (sales * quantity - discount) as TOtal_price 
FROM ORDERS_BREAKDOWN
"""
        
sql.read_sql(query, con = db_connection)


,discount,sales,quantity,TOtal_price
0,0.5,45.0,3,134.5
1,0.0,854.0,7,5978.0
2,0.0,140.0,3,420.0
3,0.5,27.0,2,53.5
4,0.5,17.0,2,33.5
...,...,...,...,...
8042,0.0,245.0,2,490.0
8043,0.5,30.0,2,59.5
8044,0.5,23.0,4,91.5
8045,0.5,108.0,3,323.5


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [176]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
query = """SELECT discount, sales, quantity, (sales * quantity - discount) as Total_price 
FROM ORDERS_BREAKDOWN
Where Total_price >100
"""
        
sql.read_sql(query, con = db_connection)


,discount,sales,quantity,Total_price
0,0.5,45.0,3,134.5
1,0.0,854.0,7,5978.0
2,0.0,140.0,3,420.0
3,0.0,90.0,3,270.0
4,0.0,207.0,4,828.0
...,...,...,...,...
6461,0.0,261.0,6,1566.0
6462,0.0,32.0,5,160.0
6463,0.0,245.0,2,490.0
6464,0.5,108.0,3,323.5


In [185]:
query = """SELECT *
FROM ORDERS_BREAKDOWN

"""
        
sql.read_sql(query, con = db_connection)



,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies
...,...,...,...,...,...,...
8042,ES-2014-3458802,"Ikea Stackable Bookrack, Traditional",0.0,245.0,2,Furniture
8043,IT-2014-3715679,"BIC Pencil Sharpener, Fluorescent",0.5,30.0,2,Office Supplies
8044,IT-2014-3715679,"Avery Binder Covers, Recycled",0.5,23.0,4,Office Supplies
8045,IT-2014-3715679,"StarTech Phone, Red",0.5,108.0,3,Technology


In [186]:
query = """
        SELECT * 
        FROM orders
        LIMIT 10;
        """
        
sql.read_sql(query, con = db_connection)

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


#### 8. Select All Customer And The Product They Have Bought

In [219]:
# Select All Customer And The Product They Have Bought

query = """SELECT ORDERS_BREAKDOWN.Product_name, orders.customer_name, ORDERS_BREAKDOWN.Order_ID
FROM ORDERS_BREAKDOWN
JOIN orders ON orders.order_ID = ORDERS_BREAKDOWN.Order_ID"""
sql.read_sql(query, con = db_connection)


,product_name,customer_name,order_id
0,"Enermax Note Cards, Premium",Eugene Moren,IT-2011-3647632
1,"Dania Corner Shelving, Traditional",Dorothy Dickinson,ES-2011-4869686
2,"Binney & Smith Sketch Pad, Easy-Erase",Arthur Prichep,ES-2011-4939443
3,"Boston Markers, Easy-Erase",Grant Thornton,IT-2011-2942451
4,"Eldon Folders, Single Width",Grant Thornton,IT-2011-2942451
...,...,...,...
8042,"Ikea Stackable Bookrack, Traditional",John Grady,ES-2014-3458802
8043,"BIC Pencil Sharpener, Fluorescent",Jim Kriz,IT-2014-3715679
8044,"Avery Binder Covers, Recycled",Jim Kriz,IT-2014-3715679
8045,"StarTech Phone, Red",Jim Kriz,IT-2014-3715679


#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [218]:
# Select Number of 'Furniture' Orders For Each Country



query = """SELECT ORDERS_BREAKDOWN.category, orders.country,count(ORDERS_BREAKDOWN.Order_ID)
FROM ORDERS_BREAKDOWN
JOIN orders ON orders.order_ID = ORDERS_BREAKDOWN.Order_ID
where ORDERS_BREAKDOWN.category = 'Furniture'
GROUP BY orders.country
"""
sql.read_sql(query, con = db_connection)

,category,country,count(ORDERS_BREAKDOWN.Order_ID)
0,Furniture,Austria,40
1,Furniture,Belgium,20
2,Furniture,Denmark,9
3,Furniture,Finland,16
4,Furniture,France,299
5,Furniture,Germany,264
6,Furniture,Ireland,12
7,Furniture,Italy,136
8,Furniture,Netherlands,76
9,Furniture,Norway,6


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [222]:
# Select Number of 'Furniture' Orders For The Country Denmark
query = """SELECT ORDERS_BREAKDOWN.category, orders.country,count(ORDERS_BREAKDOWN.Order_ID)
FROM ORDERS_BREAKDOWN
JOIN orders ON orders.order_ID = ORDERS_BREAKDOWN.Order_ID
where ORDERS_BREAKDOWN.category = 'Furniture'AND orders.country = "Denmark"
GROUP BY orders.country
"""
sql.read_sql(query, con = db_connection)

,category,country,count(ORDERS_BREAKDOWN.Order_ID)
0,Furniture,Denmark,9


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [235]:
# Select Total Sales With Discount and Without Discount for Each Country
# Select Total Sales With Discount and Without Discount for Each Country
query = """
SELECT orders.country,
SUM((ORDERS_BREAKDOWN.sales * ORDERS_BREAKDOWN.quantity) - ORDERS_BREAKDOWN.discount) AS total_price_discounted,
SUM(ORDERS_BREAKDOWN.sales * ORDERS_BREAKDOWN.quantity) As wo_discount
FROM ORDERS_BREAKDOWN 
JOIN orders ON orders.order_ID = ORDERS_BREAKDOWN.Order_ID
GROUP BY orders.country
"""

sql.read_sql(query, con = db_connection)


,country,total_price_discounted,wo_discount
0,Austria,391694.00,391694.0
1,Belgium,229829.00,229829.0
2,Denmark,34586.50,34617.0
3,Finland,82001.00,82001.0
4,France,3108445.00,3108592.0
5,Germany,2411393.15,2411483.0
6,Ireland,75673.70,75724.0
7,Italy,1177798.40,1177921.0
8,Netherlands,379525.10,379714.0
9,Norway,99878.00,99878.0


#### 11.A Select Total Quantity, Total Sales for Each Country

In [238]:
# Select Total Quantity, Total Sales for Each Country
query = """
SELECT orders.country,
SUM(ORDERS_BREAKDOWN.sales) as total_sales,
SUM(ORDERS_BREAKDOWN.quantity) as total_quantity
FROM ORDERS_BREAKDOWN 
JOIN orders ON orders.order_ID = ORDERS_BREAKDOWN.Order_ID
GROUP BY orders.country
"""

sql.read_sql(query, con = db_connection)


,country,total_sales,total_quantity
0,Austria,79382.0,973
1,Belgium,42320.0,532
2,Denmark,7763.0,204
3,Finland,20702.0,201
4,France,609683.0,7329
5,Germany,488681.0,6179
6,Ireland,15998.0,392
7,Italy,252742.0,3612
8,Netherlands,70313.0,1526
9,Norway,20529.0,261


#### 11.B Select Top 3 Country Based on Sales

In [243]:
# Select Top 3 Country Based on Sales
query = """
SELECT orders.country,
SUM(ORDERS_BREAKDOWN.sales) as total_sales,
SUM(ORDERS_BREAKDOWN.quantity) as total_quantity
FROM ORDERS_BREAKDOWN 
JOIN orders ON orders.order_ID = ORDERS_BREAKDOWN.Order_ID
GROUP BY orders.country
Order by total_sales 
limit  3
"""

sql.read_sql(query, con = db_connection)

,country,total_sales,total_quantity
0,Denmark,7763.0,204
1,Portugal,15106.0,286
2,Ireland,15998.0,392


#### 11.C Select Bottom 3 Country Based On Quantities

In [244]:
# Select Bottom 3 Country Based On Quantities

query = """
SELECT orders.country,
SUM(ORDERS_BREAKDOWN.sales) as total_sales,
SUM(ORDERS_BREAKDOWN.quantity) as total_quantity
FROM ORDERS_BREAKDOWN 
JOIN orders ON orders.order_ID = ORDERS_BREAKDOWN.Order_ID
GROUP BY orders.country
Order by total_quantity desc
limit  3
"""

sql.read_sql(query, con = db_connection)

,country,total_sales,total_quantity
0,France,609683.0,7329
1,Germany,488681.0,6179
2,United Kingdom,420497.0,4917


#### 12. Select Average Sales By Categroy For The Country 'France'

In [252]:
# Select Average Sales By Categroy For The Country 'France'
query = """
SELECT ORDERS_BREAKDOWN.category, orders.country,
AVG(ORDERS_BREAKDOWN.sales) as average_sales
FROM ORDERS_BREAKDOWN 
JOIN orders ON orders.order_ID = ORDERS_BREAKDOWN.Order_ID
where orders.country = 'France'
Group By ORDERS_BREAKDOWN.category


"""


sql.read_sql(query, con = db_connection)

,category,country,average_sales
0,Furniture,France,582.314381
1,Office Supplies,France,167.217709
2,Technology,France,595.145078


#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [262]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest
query = """
SELECT ORDERS_BREAKDOWN.category, orders.country,
AVG(ORDERS_BREAKDOWN.sales) as average_sales,
Sum(ORDERS_BREAKDOWN.sales) as total_sales
FROM ORDERS_BREAKDOWN 
JOIN orders ON orders.order_ID = ORDERS_BREAKDOWN.Order_ID
Group By ORDERS_BREAKDOWN.category, orders.country

Order by average_sales desc


"""
sql.read_sql(query, con = db_connection)

,category,country,average_sales,total_sales
0,Technology,Switzerland,902.846154,11737.0
1,Technology,Belgium,715.315789,13591.0
2,Furniture,Norway,704.000000,4224.0
3,Technology,United Kingdom,703.780992,170315.0
4,Technology,Spain,655.066667,88434.0
5,Technology,Germany,627.913183,195281.0
6,Furniture,Finland,600.125000,9602.0
7,Furniture,United Kingdom,597.761111,107597.0
8,Technology,France,595.145078,229726.0
9,Furniture,France,582.314381,174112.0


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



